# nd 原版

In [ ]:
import numpy as np
import random
tao_points = np.array([
    0, 0.045, 0.105, 0.209, 0.292, 0.396, 0.5, 0.604, 0.708, 0.794, 0.9, 1
])
x_points = np.array([
    0, 10.329, 22.755, 38.104, 55.757, 58.086, 66.722, 78.909, 106.655, 124.542, 124.553, 126.646
])

def get_x_from_tau(tau, detail_output=False):
    if tau <= tao_points[0]:
        return x_points[0]
    int_part = np.floor(tau)
    x_val = int_part * x_points[-1]
    tau = tau - int_part

    idx = np.searchsorted(tao_points, tau, side='right') - 1

    tau_start = tao_points[idx]
    tau_end = tao_points[idx + 1]
    x_start = x_points[idx]
    x_end = x_points[idx + 1]

    current_slope = (x_end - x_start) / (tau_end - tau_start)

    x_val += x_start + current_slope * (tau - tau_start)

    return x_val


# Segment lengths
segments = [225+601, 462+467]
# T range
T_min = 715
T_max = 1149
num_simulations = 1000000
count_S_greater_than_TH = 0
S_values = []
for i in range(num_simulations):
    current_S = 0
    for j, segment_length in enumerate(segments):
        T = random.randint(T_min, T_max)

        tau = segment_length / T
        X = get_x_from_tau(tau)
        current_S += X

    S_values.append(current_S)
    TH = random.uniform(335, 344) #6炮对于最左巨人行走335，对于最右巨人344

    if current_S > TH:
        count_S_greater_than_TH += 1


# 统计结果
percent = count_S_greater_than_TH / num_simulations * 100

print(count_S_greater_than_TH)
print(f'{percent:.3f}%')

# nd 改版

In [ ]:
import numpy as np

tao_points = np.array([
    0, 0.045, 0.105, 0.209, 0.292, 0.396, 0.5, 0.604, 0.708, 0.794, 0.9, 1
])
x_points = np.array([
    0, 10.329, 22.755, 38.104, 55.757, 58.086, 66.722, 78.909, 106.655, 124.542, 124.553, 126.646
])

def get_x_from_tau_vec(tau):
    # tau: shape (n,) or (n, m)
    tau = np.asarray(tau)
    int_part = np.floor(tau)
    x_val = int_part * x_points[-1]
    tau_mod = tau - int_part

    # 处理 tau_mod <= tao_points[0] 的情况
    idx = np.searchsorted(tao_points, tau_mod, side='right') - 1
    idx = np.clip(idx, 0, len(tao_points) - 2)  # 防止越界

    tau_start = tao_points[idx]
    tau_end = tao_points[idx + 1]
    x_start = x_points[idx]
    x_end = x_points[idx + 1]
    current_slope = (x_end - x_start) / (tau_end - tau_start)
    x_val += np.where(
        tau_mod <= tao_points[0],
        x_points[0],
        x_start + current_slope * (tau_mod - tau_start)
    )
    return x_val

# 向量化模拟
num_simulations = 100000000
segments = np.array([225+601, 462+467])

# 随机速度
v = np.random.uniform(0.23, 0.37, size=(num_simulations, 2))
T = np.ceil(124.1 / 0.47 / v)
tau = segments / T  # shape: (num_simulations, 2)

# 计算每段的X
X = get_x_from_tau_vec(tau)  # shape: (num_simulations, 2)
current_S = X.sum(axis=1)    # shape: (num_simulations,)

# 随机阈值
TH = np.random.randint(334, 344, size=num_simulations)

# 统计结果
count_S_greater_than_TH = np.sum(current_S > TH)
percent = count_S_greater_than_TH / num_simulations * 100

print(count_S_greater_than_TH)
print(f'{percent:.3f}%')

# 新版

In [ ]:
import numpy as np

frame_x = np.array([
    4.5,4.5,4.5,4.4,4.5,
    2.9,3,3,2.9,3,
    4.4,4.4,4.4,4.4,
    0.4,0.5,0.4,0.5,0.4,
    1.7,1.7,1.7,1.7,1.7,
    2.4,2.4,2.4,2.4,2.4,
    5.3,5.4,5.4,5.4,5.4,
    4.5,4.4,4.5,4.5,
    0,0,0,0,0,
    0.4,0.4,0.2,0.4,0.4,
])
frame_piece_n = np.array([
    5, 5, 4, 5, 5, 5, 5, 4, 5, 5
])
frame_point_n = np.concatenate(([0], frame_piece_n.cumsum()))
frame_piece_x = np.array([
    frame_x[frame_point_n[i]:frame_point_n[i+1]].sum() 
    for i in range(len(frame_piece_n))
])
frame_piece_cum_x = np.concatenate(([0], frame_piece_x.cumsum()))
N = frame_piece_n.sum()
s = frame_piece_x.sum()

def calc_x(t, v, detail_output=False):
    frame = N*0.47/s*v*t
    frame_int = np.floor(frame / N)
    frame_mod = frame % N
    piece_idx = np.searchsorted(frame_point_n, frame_mod, side='right') - 1
    frame_mod_mod = frame_mod - frame_point_n[piece_idx]
    ans = (
        s * frame_int 
        + frame_piece_cum_x[piece_idx]
        + frame_piece_x[piece_idx] * frame_mod_mod / frame_piece_n[piece_idx]
    )
    if detail_output:
        print(f'frame: {frame}, frame_int: {frame_int}, frame_mod: {frame_mod}, frame_mod_mod: {frame_mod_mod}')
        print(f'piece_idx: {piece_idx}, piece_n: {frame_piece_n[piece_idx]}, piece_x: {frame_piece_x[piece_idx]}')
        print(f's_int: {s * frame_int}, s_cum: {frame_piece_cum_x[piece_idx]}, s_piece: {frame_piece_x[piece_idx] * frame_mod_mod / frame_piece_n[piece_idx]}')
    return ans * (N+1) / N


# 向量化模拟
num_simulations = 100000
segments = np.array([225+601, 462+467])

# 随机速度
v = np.random.uniform(0.23, 0.37, size=(num_simulations, 2))
t = segments[np.newaxis, :] * np.ones((num_simulations, 2))  # shape (num_simulations, 2)

# 计算每段的X
X = calc_x(t, v)  # shape: (num_simulations, 2)
current_S = X.sum(axis=1)    # shape: (num_simulations,)

# 随机阈值
TH = np.random.randint(334, 344, size=num_simulations)

# 统计结果
count_S_greater_than_TH = np.sum(current_S > TH)
percent = count_S_greater_than_TH / num_simulations * 100

print(count_S_greater_than_TH)
print(f'{percent:.3f}%')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
T_sup = 3000
v = 0.3
t_dlt = 1

In [ ]:
t_arr = np.arange(0, T_sup, t_dlt)
x_arr = calc_x(t_arr, v)
v_arr = np.diff(x_arr)
v_arr_diff = np.abs(np.diff(v_arr))
shift_idx = np.where(v_arr_diff > 0.005 * t_dlt)[0]
shift_t = t_arr[shift_idx]
shift_t = shift_t[np.r_[(np.diff(shift_t) - t_dlt) > t_dlt*1e-6, True]]
shift_t = np.round(shift_t, 1)
print(shift_t)

T_arr = np.arange(0, T_sup, t_dlt)
T_cyc = 1 / 0.47 * s / v
ans_best_ct = []
tol = 0.02 * t_dlt
prev_ct = None
for T in T_arr:
    ct_arr = np.arange(0, T // 2 + 1, t_dlt)
    cx_arr = calc_x(ct_arr, v) + calc_x(T - ct_arr, v)

    min_idx = np.argmin(cx_arr)
    # 降噪
    if prev_ct is None: best_idx = min_idx
    else:
        cand_cts = [prev_ct, prev_ct + t_dlt]
        best_idx = None
        for ct in cand_cts:
            cidx = int(np.round(ct / t_dlt))
            if 0 <= cidx < len(ct_arr):
                if cx_arr[cidx] <= cx_arr[min_idx] + tol:
                    best_idx = cidx
                    break
        if best_idx is None:
            best_idx = min_idx
    prev_ct = ct_arr[best_idx]
    # 储存
    best_ct = ct_arr[best_idx]
    ans_best_ct.append(best_ct)

ans_best_ct = np.array(ans_best_ct)
y1 = ans_best_ct % T_cyc
y2 = (T_arr - ans_best_ct) % T_cyc

eps = 0.9 * t_dlt
# 水平段掩码（与原长度对齐）
d1 = np.abs(np.diff(y1))
d2 = np.abs(np.diff(y2))
flat1 = np.r_[d1 < eps, True]
flat2 = np.r_[d2 < eps, True] & (~flat1)
flat_union = flat1 | flat2

y0 = y1*flat1 + y2*flat2

# 插值平滑
idx = np.arange(len(y0))
if flat_union.any():
    y0 = np.interp(idx, idx[flat_union], y0[flat_union])
y0 = np.where(y0 >= T_cyc - 1, 0.0, y0)

In [ ]:
size_h = 5
ratio_v = 0.37
fig, axes = plt.subplots(
    2, 2, figsize=(size_h * (T_sup / T_cyc + ratio_v), size_h * (1 + ratio_v)), 
    gridspec_kw={
        "width_ratios": [T_sup / T_cyc, ratio_v], 
        "height_ratios": [1, ratio_v],
        "wspace": 0.1, "hspace": 0.1},

)

# 1-1 最优垫材时机 vs 总时机
ax = axes[0, 0]
ax.plot(T_arr, y0, color='b')
ax.plot(T_arr, y1, alpha=0.3, color='g')
ax.plot(T_arr, y2, alpha=0.3, color='y')
ax.set_ylim(0, T_cyc)
ax.set_xlim(0, T_sup)
for y in shift_t[shift_t <= T_cyc]:
    if np.abs(y0 - y).min() > 1: continue
    ax.hlines(y=y, xmin=0, xmax=T_sup, colors='m', linestyles='dashed')
    ax.text(x=T_sup*0.0, y=y+5, s=f'{y}', color='m')
for i in range(1, int(T_sup // T_cyc) + 1):
    ax.vlines(x=i*T_cyc, ymin=0, ymax=T_cyc, colors='r', linestyles='dashed')

# 2-1 周期行走速度
ax = axes[1, 0]
ax.plot(T_arr[:-1], v_arr, color='k')
ax.set_xlim(0, T_sup)
for i in range(1, int(T_sup // T_cyc) + 1):
    ax.vlines(x=i*T_cyc, ymin=0, ymax=v_arr.max()*1.1, colors='r', linestyles='dashed')

# 1-2 单周期行走速度
ax = axes[0, 1]
ax.plot(v_arr, T_arr[:-1], color='k')
ax.invert_xaxis()
ax.set_ylim(0, T_cyc)
for y in shift_t[shift_t <= T_cyc]:
    if np.abs(y0 - y).min() > 1: continue
    ax.hlines(y=y, xmin=0, xmax=v_arr.max()*1.1, colors='m', linestyles='dashed')

In [ ]:
calc_x(359, 0.37) + calc_x(508, 0.37), calc_x(566, 0.37) + calc_x(187, 0.37)

In [ ]:
359 + 508, 566 + 187

In [ ]:
std = pd.read_csv('../output.csv')['value'].values
sim = std[0] - calc_x(np.arange(0, len(std), 1), 0.23)

In [ ]:
plt.plot(- np.diff(sim))

In [ ]:
plt.plot(sim - std)

In [ ]:
plt.plot(std, label='Standard')
plt.plot(sim, label='Simulation')